# Sustainability `data pipeline` 


In [2]:
import sqlite3
import pandas as pd
import plotly.express as px

In [6]:
df_sust = pd.read_excel('data/xcl/data_xls_sustainability.xlsx',usecols=['Region','City','GFA (m²)','Project','Name','Rating System','Level','Status'])

df_sust = df_sust[df_sust['Name'] != '-']
df_sust['GFA (m²)'] = pd.to_numeric(df_sust['GFA (m²)'], errors='coerce', downcast='integer')
df_sust.dropna(subset=['Name'], inplace=True)
df_sust.rename(columns={'Project': 'Project Number'},inplace=True)
df_sust.head(3)

,Region,City,GFA (m²),Project Number,Name,Rating System,Level,Status
0,Europe,Vilnius,19200.0000,13045-NB,14A Konstitucijos Ave,BREEAM,Outstanding,Design
1,America,Toronto,134995.2707,11370-00,30 Bay Street Toronto,LEED,Platinum,Design
2,Australia,Sydney,306466.0000,11360-00,International Towers Barangaroo,Green Star,6 Star,Built


In [8]:
df_types = pd.read_csv('data/csv/data_project_types.csv')
df_types.plot()


### Merge project types with Ratings

In [9]:
filtered_types = df_types[['Project Number','Project Type']]
sust_by_type = pd.merge(df_sust,filtered_types,on='Project Number',).sort_values('Project Type')
sust_by_type.head(3)

,Region,City,GFA (m²),Project Number,Name,Rating System,Level,Status,Project Type
39,Canada,Toronto,20000.0,6070,St Lawrence Market North,Toronto Green Standard,Tier 1,Under-construction,Civic
42,Europe,Lens,NaN,10620,Louvre Conservation Centre (Centre de Conserva...,NaN,NaN,Built,Cultural & Leisure
35,America,Colombia,11283.0,12235,Centro de Convenciones Bogota,LEED,Platinum,Design,Cultural & Leisure


---
### Analyse the `Data`

In [24]:
# Group the data by 'Rating System' and count the number of projects in each group
filtered_missing = df_sust[df_sust['Rating System'] != 'Missing']
rating_counts = filtered_missing['Rating System'].value_counts().reset_index()
rating_counts.columns = ['Rating System', 'Number of Projects']

# Create a bar chart using Plotly Express
fig = px.bar(rating_counts, x='Rating System', y='Number of Projects', title='Projects per Rating System',template='plotly_dark')
fig.show()

In [22]:
# Group the data by 'Rating System' and count the number of projects in each group
filtered_missing = df_sust[df_sust['Rating System'] != 'Missing']
rating_counts = filtered_missing['Rating System'].value_counts().reset_index()
rating_counts.columns = ['Rating System', 'Number of Projects']

# Create a bar chart using Plotly Express
fig = px.bar(rating_counts, x='Rating System', y='Number of Projects', title='Projects per Rating System',template='plotly_dark')
fig.show()

---
### Export to SQLite DB

In [25]:
conn = sqlite3.connect('db_project_zero.sqlite')
sust_by_type.to_sql('data/sqlite/sustainability', conn, if_exists='replace', index=False)

43

In [21]:
pd.read_sql('select * from sustainability', conn).head(10)

,Region,City,GFA (m²),Project Number,Name,Rating System,Level,Status,Project Type
0,Canada,Toronto,20000.0,6070.0,St Lawrence Market North,Toronto Green Standard,Tier 1,Under-construction,Civic
1,Europe,Lens,NaN,10620.0,Louvre Conservation Centre (Centre de Conserva...,None,None,Built,Cultural & Leisure
2,America,Colombia,11283.0,12235.0,Centro de Convenciones Bogota,LEED,Platinum,Design,Cultural & Leisure
3,UK,London,NaN,3690.0,Mossbourne Community Academy,Missing,Missing,Built,Education
4,Italy,Venice,2500.0,11230.0,H FARM,LEED,Gold,Built,Education
5,UK,Speyside,14800.0,6770.0,Macallan Distillery,None,None,Built,Industrial
6,Europe,Paris,NaN,10550.0,Les Messageries - Gare de Lyon Daumesnil maste...,None,None,Design,Masterplanning
7,Europe,Paris,NaN,12005.0,Maine-Montparnasse masterplan,None,None,Design,Masterplanning
8,Asia,China,88966.0,12900.0,Nanjing Yuhua,China Green Building,2 stars,Design,Mixed Use
9,Middle East,Dubai,NaN,13230.0,Dubai Water Canal Peninsula,Missing,Missing,Design,Mixed Use
